In [14]:
import pickle
import string
import pandas as pd
from os import listdir

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [18]:
images_folder = 'D:/datasets/flickr-30k-images'
curr_folder = "D:/YandexDisk/datasets/"

path_captions = curr_folder + "captions-ru-no-puncts.csv"
path_features = curr_folder + "features.pkl"
path_vocabulary = curr_folder + "vocabulary.pkl"

path_train = curr_folder + "captions-ru-test.csv"
path_val = curr_folder + "captions-ru-val.csv"
path_test = curr_folder + "captions-ru-test.csv"

# Подготовка данных к обучению

– Каждое слово с маленькой буквы
– Удалить знаки препинания
– Только буквы

In [7]:
df = pd.read_csv(path_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,два молодых парня с лохматыми волосами смотрят...
1,1000092795.jpg,1,два молодых белых самца находятся снаружи возл...
2,1000092795.jpg,2,двое мужчин в зеленых рубашках стоят во дворе
3,1000092795.jpg,3,мужчина в синей рубашке стоящий в саду
4,1000092795.jpg,4,два друга наслаждаются временем проведенным вм...


In [7]:
def clean_captions(data):
    table = str.maketrans('', '', string.punctuation)

    for idx in data.index:
        curr_capt = data.iat[idx, 2]
        curr_capt = curr_capt.split()
        curr_capt = [word.lower() for word in curr_capt]
        curr_capt = [word.translate(table) for word in curr_capt]
        curr_capt = [word for word in curr_capt if word.isalpha()]

        data.iat[idx, 2] = ' '.join(curr_capt)

In [8]:
clean_captions(df)
df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,два молодых парня с лохматыми волосами смотрят...
1,1000092795.jpg,1,два молодых белых самца находятся снаружи возл...
2,1000092795.jpg,2,двое мужчин в зеленых рубашках стоят во дворе
3,1000092795.jpg,3,мужчина в синей рубашке стоящий в саду
4,1000092795.jpg,4,два друга наслаждаются временем проведенным вм...


In [9]:
df.to_csv(path_captions, sep='|', encoding='utf-8', index=False)

# Словарь

In [9]:
def to_vocabulary(data):
    vocab = set()

    for idx in df.index:
        vocab.update(data.iat[idx, 2].split())

    return vocab

In [10]:
vocabulary = to_vocabulary(df)
print('размер словаря ... %d' % len(vocabulary))

размер словаря ... 31391


In [19]:
with open(path_vocabulary,'wb') as f:
    pickle.dump(vocabulary, f)

In [20]:
with open(path_vocabulary,'rb') as f:
    loaded_set = pickle.load(f)

# Извлечение признаков

In [ ]:
def extract_features(directory):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    print(model.summary())

    features = dict()
    for name in listdir(directory):
        filename = directory + '/' + name

        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)

        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature

    return features

In [7]:
%%time
features = extract_features(images_folder)
print('extracted features ... %d' % len(features))
pickle.dump(features, open(path_features, 'wb'))

553467904/553467096 [==============================] - 241s 0us/step

# Разбить набор на три части

In [13]:
df = pd.read_csv(path_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,два молодых парня с лохматыми волосами смотрят...
1,1000092795.jpg,1,два молодых белых самца находятся снаружи возл...
2,1000092795.jpg,2,двое мужчин в зеленых рубашках стоят во дворе
3,1000092795.jpg,3,мужчина в синей рубашке стоящий в саду
4,1000092795.jpg,4,два друга наслаждаются временем проведенным вм...


In [14]:
n = len(df)

train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_df.to_csv(path_train, sep='|', encoding='utf-8', index=False)
val_df.to_csv(path_val, sep='|', encoding='utf-8', index=False)
test_df.to_csv(path_test, sep='|', encoding='utf-8', index=False)

# Добавить начальные и конечные строки startseq и endseq

In [28]:
def add_start_end_tags(data):
    for idx in data.index:
        curr_str = data.iat[idx, 2]
        data.iat[idx, 2] = 'startseq ' + curr_str + ' endseq'

In [29]:
add_start_end_tags(train_df)
add_start_end_tags(val_df)
add_start_end_tags(test_df)

train_df.to_csv(path_train, sep='|', encoding='utf-8', index=False)
val_df.to_csv(path_val, sep='|', encoding='utf-8', index=False)
test_df.to_csv(path_test, sep='|', encoding='utf-8', index=False)

In [38]:
test_df.iat[0, 2]

'startseq мужчина в коричневокрасном костюме держит струнный инструмент и смотрит налево перед фиолетоворозовым фоном endseq'